In [5]:
import pymongo
print ("pymongo imported successfully")

pymongo imported successfully


In [6]:
username = "aacuser"
password = "SNHU1234"
hostname = "nv-desktop-services.apporto.com"
port = 30291
auth_source = "AAC"

client = pymongo.MongoClient(f"mongodb://{username}:{password}@{hostname}:{port}/{auth_source}")

db = client[auth_source]

In [7]:
db.list_collection_names()

['animals']

In [8]:
!pip install jupyter-dash
print ("dash installed successfully")

Defaulting to user installation because normal site-packages is not writeable
dash installed successfully


In [35]:
# Configure the necessary Python module imports
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl  # Import Dash Leaflet for the map component
from animalShelter import AnimalShelter
import pandas as pd
import base64

# Initialize the AnimalShelter object
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

# Retrieve all animals from the MongoDB collection
animals_data = shelter.read()

# Convert the cursor object to a DataFrame
df = pd.DataFrame(list(animals_data))

# Load the image file
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

# Get unique animal types for dropdown options
animal_types = df['animal_type'].unique()

# Dashboard Layout / View
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}')),
    html.Center(html.B(html.H2('Animal Shelter Ban Jimmy Goad'))),
    html.Hr(),
    dcc.Dropdown(
        id='animal-type-dropdown',
        options=[{'label': animal_type, 'value': animal_type} for animal_type in animal_types],
        value='Cat',  # Default value
        clearable=False,
        style={'width': '50%'}
    ),
    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,  # Set the number of rows per page
        style_table={'height': '400px', 'overflowY': 'auto', 'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'rgb(240,230,230)',
            'fontWeight': 'bold'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        tooltip={i: {'value': i, 'use_with': 'both'} for i in df.columns},
        tooltip_delay=0,
        tooltip_duration=None,
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[0],  # Select the first row by default
    ),
    html.Div(id='map-container')
])

# Callback to update the data table based on selected animal type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('animal-type-dropdown', 'value')]
)
def update_data_table(animal_type):
    filtered_df = df[df['animal_type'] == animal_type]
    return filtered_df.to_dict('records')

# Callback to update the map based on the selected row in the data table
@app.callback(
    Output('map-container', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    try:
        if index is None or len(index) == 0:
            return html.Div("No row selected")

        row = index[0]
        selected_row_data = viewData[row]

        latitude = selected_row_data.get('location_lat')
        longitude = selected_row_data.get('location_long')
        animal_name = selected_row_data.get('name')

        if latitude is None or longitude is None:
            return html.Div(f"No geolocation data available for selected row (Animal Name: {animal_name})")

        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[latitude, longitude], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(position=[latitude, longitude],
                                 children=[
                                     dl.Tooltip(animal_name),
                                     dl.Popup([
                                         html.H1("Animal Name"),
                                         html.P(animal_name)
                                     ])
                                 ])
                   ])
        ]
    except Exception as e:
        print("Error updating map:", e)
        return html.Div("An error occurred while updating the map.")

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:10919/
